![MuJoCo banner](https://raw.githubusercontent.com/google-deepmind/mujoco/main/banner.png)

# <h1><center>Tutorial  <a href="https://colab.research.google.com/github/google-deepmind/mujoco_warp/blob/main/notebooks/tutorial.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="140" align="center"/></a></center></h1>

This notebook provides an introductory tutorial for [**MuJoCo Warp (MJWarp)**](https://github.com/google-deepmind/mujoco_warp/), an implementation of [MuJoCo](https://github.com/google-deepmind/mujoco) written using [NVIDIA Warp](https://github.com/NVIDIA/warp).

This tutorial assumes a basic understanding of MuJoCo.  See the [MuJoCo Basics colab](https://colab.sandbox.google.com/github/google-deepmind/mujoco/blob/main/python/tutorial.ipynb) for an introduction to MuJoCo.

**A Colab runtime with GPU acceleration is required.** If you're using a CPU-only runtime, you can switch using the menu "Runtime > Change runtime type".










# All imports

In [1]:
!uv pip install "git+https://github.com/google-deepmind/mujoco_warp.git"
!uv pip install mediapy

import os
import subprocess
import mediapy as media
import numpy as np
import warp as wp

# Set up GPU rendering.
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Configuring MuJoCo for GPU rendering, setting ', end="")
%env MUJOCO_GL=egl

# avoid warp output in cells
wp.config.quiet = True

try:
  print('Validating MuJoCo Warp installation: ', end="")
  import mujoco
  import mujoco_warp as mjw
  mjw.put_model(mujoco.MjModel.from_xml_string('<mujoco/>'))
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('success.')

Using Python 3.12.12 environment at: /usr
Resolved 12 packages in 305ms
Audited 12 packages in 0.49ms
Using Python 3.12.12 environment at: /usr
Audited 1 package in 490ms
Configuring MuJoCo for GPU rendering, setting env: MUJOCO_GL=egl
Validating MuJoCo Warp installation: success.


# Introduction to MuJoCo Warp

MJWarp is designed for simulating many worlds in parallel on NVIDIA GPUs, enabling rapid robot training and evaluation in physically accurate simulated environments.

MJWarp initializes directly from standard MuJoCo data structures, so we begin by loading a MuJoCo model:

In [27]:
mj_model = mujoco.MjModel.from_xml_string("""
<mujoco>
  <option timestep="0.01666"/> <!-- ~60 hz -->
  <worldbody>
    <light name="top" pos="0 0 1"/>
    <body name="box_and_sphere" euler="0 0 -30">
      <joint name="swing" type="hinge" axis="1 -1 0" pos="-.2 -.2 -.2"/>
      <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
      <geom name="green_sphere" pos="-.2 -.2 -.2" size=".1" rgba="0 1 0 1"/>
    </body>
  </worldbody>
</mujoco>
""")

Let's run the simulation in MuJoCo and render a trajectory. This example is taken from the [MuJoCo tutorial](https://colab.sandbox.google.com/github/google-deepmind/mujoco/blob/main/python/tutorial.ipynb).

In [28]:
renderer = mujoco.Renderer(mj_model)

frames = []
mj_data = mujoco.MjData(mj_model)
for _ in range(3 * 60):  # 3 second trajectory
  mujoco.mj_step(mj_model, mj_data)
  renderer.update_scene(mj_data)
  frames.append(renderer.render())

# Simulate and display video.
media.show_video(frames)

This browser does not support the video tag.


Next we take the MuJoCo `MjModel` and `MjData`, and place them on the GPU using MJWarp.

In [30]:
model = mjw.put_model(mj_model)
data = mjw.put_data(mj_model, mj_data)

Below, we print the `qpos` from MuJoCo and MJWarp. Notice that the `qpos` for the `mjData` is a numpy array living on the CPU, while the `qpos` for `mjw.Data` is a Warp array living on the GPU device.

In [5]:
print(mj_data.qpos, type(mj_data.qpos))
print(data.qpos, type(data.qpos), data.qpos.device)

[-0.07676552] <class 'numpy.ndarray'>
[[-0.07676552]] <class 'warp._src.types.array'> cuda:0


Now let's run the same exact simulation on the GPU device using MJWarp!

In the example below, we call `mjw.step` instead of `mujoco.mj_step`. For each frame, we convert the `mjw.Data` back to `mjData` so that we can use the MuJoCo renderer.


In [31]:
mjw.reset_data(model, data)
frames = []
for _ in range(3 * 60):
  mjw.step(model, data)
  mjw.get_data_into(mj_data, mj_model, data)
  renderer.update_scene(mj_data)
  frames.append(renderer.render())

media.show_video(frames)

This browser does not support the video tag.


The cell above executing on GPU takes longer than the cell running the same simulation on CPU.  This is because:

* Warp performs just-in-time (JIT) runtime compilation of Python functions to CUDA kernel-level code.  This compilation is automatically cached, so the second call to `mjw.step` is much faster.
* `mjw.step` dispatches hundreds of Warp function calls to the GPU serially, each with a small dispatch overhead cost.

Knowing this, we can improve the performance in the following ways:

* Where possible, we warm up the JIT cache ahead of time, and use the cache across function calls or even across process.
* To reduce kernel dispatch overhead, we record the sequence of CUDA operations in `mjw.step`, so that they can be [executed on GPU as a single graph](https://nvidia.github.io/warp/modules/concurrency.html#synchronization-and-graph-capture).  To do this, we use Warp's `ScopedCapture` context manager.

Finally, to fully leverage MJWarp's design, we will simulate not one, but 4096 worlds in parallel:

In [33]:
data = mjw.put_data(mj_model, mj_data, nworld=4096)

with wp.ScopedCapture() as capture:
  mjw.step(model, data)

mjw.reset_data(model, data)
frames = []
for _ in range(3 * 60):
  wp.capture_launch(capture.graph)
  # fetch the first world back onto CPU for rendering
  mjw.get_data_into(mj_data, mj_model, data)
  renderer.update_scene(mj_data)
  frames.append(renderer.render())

media.show_video(frames)

This browser does not support the video tag.


MJWarp not only operates on batches of `Data` - it also operate on batches of `Model` fields.

When MJWarp encounters a batched `Model` field, it distributes the field's values evenly across worlds.  Assigning worlds different model properties is a simple way to perform [domain randomization](https://lilianweng.github.io/posts/2019-05-05-domain-randomization/) in machine learning workloads.


As an example, the following code assigns reverse gravity to half of the 4096 worlds:

In [35]:
data = mjw.put_data(mj_model, mj_data, nworld=4096)
gravity = np.array([0, 0, -9.81])
model.opt.gravity = wp.array([gravity, -gravity], dtype=wp.vec3f)
mjw.step(model, data)

# MuJoCo Warp for robotics research

MJWarp is integrated into some existing toolkits for training robots using machine learning:

* [MuJoCo Playground](https://github.com/google-deepmind/mujoco_playground) is
a comprehensive suite of GPU-accelerated environments for robot learning research and sim-to-real.  It is written in JAX and integrates MJWarp for physics.

* [Isaac Lab](https://github.com/isaac-sim/IsaacLab/tree/feature/newton) is a GPU-accelerated, open-source framework designed by NVIDIA for robotics research.  It has beta support for MJWarp through a new physics platform called [Newton](https://developer.nvidia.com/newton-physics).

* [mjlab](https://github.com/mujocolab/mjlab) combines Isaac Lab's proven API with MJWarp to provide lightweight, modular abstractions for RL robotics research and sim-to-real deployment.

👋 Tawa pona!